In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.graph_objs as gp

In [2]:
vax_df = pd.read_csv('datasets/dados_vacinacao.csv')

In [3]:
vax_df.head(1)

,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_nmMunicipio,paciente_endereco_cep,estabelecimento_municipio_nome,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,paciente_id
0,44,1977-04-18,M,1,BRANCA,ARAPIRACA,57510,PATOS,912,Médico,9,Trabalhadores de Saúde,2021-11-11,Reforço,87,COVID-19 PFIZER - COMIRNATY,f9495cb04daf5adb495d22c9c88b31883f191fb762ab27...


<p>De início, será feita uma limpeza nos dados excluindo os dados que podem ser provenientes de erros no Dataset original. Tais como: Idades negativas, muitas idades maiores do que 105 anos (corte arbitrário) e pessoas que aparecem como tendo sido vacinadas antes de 2021.<p>
Para isso, começaremos convertendol as datas que estão em string para datetime

In [4]:
vax_df.vacina_dataAplicacao = pd.to_datetime(vax_df.vacina_dataAplicacao)

In [5]:
drop_this = vax_df.loc[(vax_df.paciente_idade > 110) | (vax_df.paciente_idade < 5) | (vax_df.vacina_dataAplicacao < pd.Timestamp(2021, 1, 17))]

drop_this

,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_nmMunicipio,paciente_endereco_cep,estabelecimento_municipio_nome,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,paciente_id
15030,121,1899-12-30,F,99,SEM INFORMACAO,PICUI,NaN,PICUI,917,Profissionais e Auxiliares de limpeza,9,Trabalhadores de Saúde,2021-01-28,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,f1c3b1a19f65bc23f26fd66ad23411e8ce0583295b8471...
38305,2,2019-09-16,F,99,SEM INFORMACAO,GUARULHOS,7142,SOUSA,207,Pessoas de 5 a 11 anos,0,NaN,2022-03-28,2ª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,2148017dbfa2015a4ed271aac5dcadeca60557c279e51c...
39339,0,1974-11-16,F,99,SEM INFORMACAO,CAMPINA GRANDE,58401,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,1975-02-20,1ª Dose,88,COVID-19 JANSSEN - Ad26.COV2.S,046d325facf10e1f13f78e4e9195cb2dc27953e5075484...
41754,0,1986-04-20,F,99,SEM INFORMACAO,CAMPINA GRANDE,58401,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,1986-04-20,Reforço,87,COVID-19 PFIZER - COMIRNATY,0469ebe1ad08f5040adca17ec248430e5fbf112b749c47...
46665,-8,1983-02-28,M,99,SEM INFORMACAO,MACEIO,None,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,1974-12-05,2ª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,22618b1bff1b85fa4c238413d60891763eb37fcacf7cdf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727528,0,1962-07-08,M,99,SEM INFORMACAO,BAYEUX,58305,JOAO PESSOA,201,Pessoas de 18 a 64 anos,2,Faixa Etária,1962-06-20,1ª Dose,87,COVID-19 PFIZER - COMIRNATY,e41bf7e6e369aa29e6d26ec77d1d8daa0f60a924a4cbcc...
2733405,111,1910-01-12,F,3,PARDA,JOAO PESSOA,58078,JOAO PESSOA,205,Pessoas de 80 anos ou mais,2,Faixa Etária,2021-02-07,1ª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,dbe3f058be16f383a8392fbb4f89681caf441363366bfa...
2791852,0,1999-02-17,M,3,PARDA,JOAO PESSOA,58034,JOAO PESSOA,507,Policial Militar,5,Forças de Segurança e Salvamento,1999-02-17,2ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,7f5626eb38d0699cd1c44aef7dfc61bc02ab1589d35433...
2798130,0,1980-04-24,M,99,SEM INFORMACAO,CAMPINA GRANDE,58107,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,1980-04-24,2ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,dc02c8fb8b211283163a6d8680bbb81216e0831436bf94...


A data escolhida para ser o ponto de corte foi 17 de janeiro de 2021, o dia em que a primeira brasileira a ser vacinada recebeu a primeira vacina aplicada em território nacional.
Essas 153 ocorrências marcadas acima não serão interessantes para as análises a seguir que envolvem esse DataSet. 

In [6]:
vax_df.drop(drop_this.index, inplace = True)

In [7]:
len(vax_df.paciente_id)

2806607

### PIRÂMIDE ETÁRIA

Com o objetivo de medir a adesão da vacina até a data mais recente do DataFrame, as linhas que correspondem aos vacinados com menos de 25 anos serão excluídas, para que a comparação com a pirâmide etária do estado faça sentido.<br><br>
Fontes:                       AJEITAR DEPOIS E DEIXAR BONITO
<br>
https://g1.globo.com/pb/paraiba/noticia/2021/08/15/joao-pessoa-vacina-pessoas-com-21-anos-ou-mais-contra-a-covid-19-nesta-segunda-feira-16.ghtml
<br>
https://g1.globo.com/pb/paraiba/noticia/2021/08/09/campina-grande-vacina-contra-covid-19-pessoas-a-partir-dos-26-anos-nesta-terca-feira-10.ghtml

Além disso, para contar de forma correta a quantidade de vacinados por faixa etária, os pacientes que aparecem mais de uma vez serão retirados. AJEITAR DEPOIS


In [8]:
etaria_df = vax_df.loc[vax_df.paciente_idade >= 25].drop_duplicates(subset= 'paciente_id')

In [9]:
etaria_df.loc[etaria_df.paciente_idade > 25]

,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_nmMunicipio,paciente_endereco_cep,estabelecimento_municipio_nome,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,paciente_id
0,44,1977-04-18,M,1,BRANCA,ARAPIRACA,57510,PATOS,912,Médico,9,Trabalhadores de Saúde,2021-11-11,Reforço,87,COVID-19 PFIZER - COMIRNATY,f9495cb04daf5adb495d22c9c88b31883f191fb762ab27...
2,39,1982-01-23,F,99,SEM INFORMACAO,CAMPINA GRANDE,58410,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-07-16,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,0ffaed327aac7fe8abf0b35471b0b1fa60f9fc48ff7d9c...
3,33,1988-07-20,M,2,PRETA,BAYEUX,58110,BAYEUX,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-07-23,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,fe78cee14dce3436024728b1d760439861021207a32712...
4,27,1994-03-22,M,3,PARDA,JOAO PESSOA,58058,JOAO PESSOA,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-12-07,2ª Dose,87,COVID-19 PFIZER - COMIRNATY,470944c18359d6a1fea2a53194d29bfdb7ce1d21b47979...
6,64,1957-03-24,F,2,PRETA,CAMPINA GRANDE,NaN,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-05-19,2ª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,0f4979537aa2e0a219d6cc6444f63dd9822320d9de4035...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806772,60,1961-04-10,F,1,BRANCA,CAMPINA GRANDE,58407,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-04-19,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,9488b77754e8cd63ae0e57a69e77072d6d9b552d824b9c...
2806775,50,1971-02-11,F,99,SEM INFORMACAO,JOAO PESSOA,58085,JOAO PESSOA,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-06-25,1ª Dose,88,COVID-19 JANSSEN - Ad26.COV2.S,7d9c473fc1707d8c640893186609811a91cf0c2abfab99...
2806776,39,1981-08-29,M,2,PRETA,SERRA BRANCA,58580,SERRA BRANCA,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-07-09,1ª Dose,87,COVID-19 PFIZER - COMIRNATY,0f2de66558d8c8a12ec5347bbb25a89d78bc6141be5e94...
2806787,66,1954-09-06,F,3,PARDA,REMIGIO,58398,REMIGIO,202,Pessoas de 65 a 69 anos,2,Faixa Etária,2021-03-24,1ª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,a96a8f1ca4d357e0faa72dc7f83ab56556a94064ad6e91...


In [10]:
pir_list = []

In [11]:
for i in range(25, 101, 5):
    pir_dict = {}

    if i == 100:
        pir_dict['Age'] = '100+'
        gender_count = etaria_df.loc[etaria_df.paciente_idade >= 100].value_counts('paciente_enumSexoBiologico')

    else:
        ind = f'{i}-{i + 4}'
        pir_dict['Age'] = ind
        gender_count = etaria_df.loc[etaria_df.paciente_idade.isin(range(i, i + 5))].value_counts('paciente_enumSexoBiologico')

    pir_dict['M'] = gender_count['M']
    pir_dict['F'] = gender_count['F']

    pir_list.append(pir_dict)

In [12]:
pir_df = pd.DataFrame(pir_list)

In [13]:
pir_df.columns = ['idade', 'masculino', 'feminino']

In [14]:
pir_df

,idade,masculino,feminino
0,25-29,86264,96341
1,30-34,85871,98088
2,35-39,90602,101534
3,40-44,85142,96105
4,45-49,75484,86358
5,50-54,67661,79767
6,55-59,63381,74753
7,60-64,52513,62617
8,65-69,41653,51728
9,70-74,33147,43038


In [15]:
y_age = pir_df.idade
x_M = pir_df.masculino
x_F = pir_df.feminino * -1

In [16]:
fig = gp.Figure()
  

fig.add_trace(gp.Bar(y= y_age, x = x_M, 
                     name = 'Masculino', 
                     orientation = 'h'))
  
fig.add_trace(gp.Bar(y = y_age, x = x_F,
                     name = 'Feminino', orientation = 'h'))
  

fig.update_layout(title = 'Vacinados por gênero',
                 title_font_size = 22, barmode = 'relative',
                 bargap = 0.0, bargroupgap = 0,
                xaxis = dict(
                              title = 'Vacinados em milhares de habitantes',
                              title_font_size = 14)
                 )
  
fig.show()

In [17]:
md_masc = np.mean(pir_df.masculino)
std_masc = np.std(pir_df.masculino)

pir_df['norm_masc'] = norm.cdf(pir_df.masculino, loc = md_masc, scale = std_masc)

In [18]:
md_fem = np.mean(pir_df.feminino)
std_fem = np.std(pir_df.feminino)

pir_df['norm_fem'] = norm.cdf(pir_df.feminino, loc = md_fem, scale = std_fem)

In [22]:
pir_df['total'] = pir_df.masculino + pir_df.feminino

In [24]:
med_total = np.mean(pir_df.total)
std_total = np.std(pir_df.total)

In [27]:
pir_df.norm_masc = norm.cdf(pir_df.masculino, loc = med_total, scale = std_total)
pir_df.norm_fem = norm.cdf(pir_df.feminino, loc = med_total, scale = std_total)

In [31]:
pir_df

,idade,masculino,feminino,norm_masc,norm_fem,total
0,25-29,86264,96341,0.422813,0.480239,182605
1,30-34,85871,98088,0.420598,0.490271,183959
2,35-39,90602,101534,0.447406,0.510071,192136
3,40-44,85142,96105,0.416496,0.478884,181247
4,45-49,75484,86358,0.363177,0.423343,161842
5,50-54,67661,79767,0.321804,0.386564,147428
6,55-59,63381,74753,0.300031,0.359234,138134
7,60-64,52513,62617,0.247982,0.296216,115130
8,65-69,41653,51728,0.201217,0.244418,93381
9,70-74,33147,43038,0.168578,0.206870,76185


In [32]:
y_age = pir_df.idade
x_M = pir_df.norm_masc
x_F = pir_df.norm_fem * -1

In [33]:
fig = gp.Figure()
  

fig.add_trace(gp.Bar(y= y_age, x = x_M, 
                     name = 'Masculino', 
                     orientation = 'h'))
  

fig.add_trace(gp.Bar(y = y_age, x = x_F,
                     name = 'Feminino', orientation = 'h'))
  

fig.update_layout(title = 'Vacinados por gênero normalizado',
                 title_font_size = 22, barmode = 'relative',
                 bargap = 0.0, bargroupgap = 0,
                xaxis = dict(
                              title = 'Vacinados em milhares de habitantes',
                              title_font_size = 14)
                 )
  
fig.show()